In [ ]:
#!pip install -U "pycaret[full]"
#!pip install optuna
#!pip install backtesting

**После pip install -U "pycaret[full]" нужен софт RESET!!!**

<img src="https://s4.gifyu.com/images/animation_200_knjdldvj.gif" width="200" height="200">

План:
1. В качестве фичей используем результаты последних 5 матчей (PCA), пробуем короткими трендами найти профитные ставки
2. Таргет переоценена ли ставка (смотри Introduction notebook)
3. Использую pycaret c кастомной score function, ищем лучшее соотношение
4. Используем фреймворк backtest для бэктеста модели

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from scipy.stats import poisson
import math

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

<img src="https://cdn-static.dagospia.com/img/patch/06-2018/italia-campione-del-mondo-2006-1027177.gif" width="500" height="300">

# **Готовим датасет**
(см. Introduction notebook)

In [ ]:
calcio = pd.read_csv('/kaggle/input/sports-betting/calcio_italiano_risultati_scomesse.csv')
calcio = calcio.loc[calcio.week > 5] #Отрезаем ранние игры без накопленной статистики
calcio_columns = list(calcio.columns) 
calcio_columns[9] = 'team1_draw' #Исправляем название
calcio.columns = calcio_columns

calcio['K1'] = round(calcio.P1 * (1 / calcio.P1 + 1 / calcio.PD + 1/calcio.P2) / 1.05, 2)
calcio['KD'] = round(calcio.PD * (1 / calcio.P1 + 1 / calcio.PD + 1/calcio.P2) / 1.05, 2)
calcio['K2'] = round(calcio.P2 * (1 / calcio.P1 + 1 / calcio.PD + 1/calcio.P2) / 1.05, 2)
calcio['R1'] = 1 / calcio.P1 / (1 / calcio.P1 + 1 / calcio.PD + 1/calcio.P2)
calcio['RD'] = 1 / calcio.PD / (1 / calcio.P1 + 1 / calcio.PD + 1/calcio.P2)
calcio['R2'] = 1 / calcio.P2 / (1 / calcio.P1 + 1 / calcio.PD + 1/calcio.P2)

calcio = calcio.loc[(calcio.R1 > 0.15) & (calcio.R1 <0.85)]
calcio = calcio.iloc[:-10,:]

calcio['home_goals_fact'] = calcio.team1_goal #фактический результат
calcio['away_goals_fact'] = calcio.team2_goal
calcio.loc[calcio['home_goals_fact'] == 0, 'home_goals_fact'] = 0.25 #небольшой коэффициент к 0, 
calcio.loc[calcio['away_goals_fact'] == 0, 'away_goals_fact'] = 0.175 #чтобы получить распределение Пуассона
#Для каждой команды расчитаем вероятность забить n -мячей
calcio['f_home_0'] = poisson.pmf(0.25, mu=calcio.home_goals_fact, loc=0) 
calcio['f_away_0'] = poisson.pmf(0.175, mu=calcio.away_goals_fact, loc=0)
calcio['f_home_1'] = poisson.pmf(1, mu=calcio.home_goals_fact, loc=0)
calcio['f_away_1'] = poisson.pmf(1, mu=calcio.away_goals_fact, loc=0)
calcio['f_home_2'] = poisson.pmf(2, mu=calcio.home_goals_fact, loc=0)
calcio['f_away_2'] = poisson.pmf(2, mu=calcio.away_goals_fact, loc=0)
calcio['f_home_3'] = poisson.pmf(3, mu=calcio.home_goals_fact, loc=0)
calcio['f_away_3'] = poisson.pmf(3, mu=calcio.away_goals_fact, loc=0)
calcio['f_home_4'] = poisson.pmf(4, mu=calcio.home_goals_fact, loc=0)
calcio['f_away_4'] = poisson.pmf(4, mu=calcio.away_goals_fact, loc=0)
calcio['f_home_5'] = poisson.pmf(5, mu=calcio.home_goals_fact, loc=0)
calcio['f_away_5'] = poisson.pmf(5, mu=calcio.away_goals_fact, loc=0)
calcio['f_home_6'] = poisson.pmf(6, mu=calcio.home_goals_fact, loc=0)
calcio['f_away_6'] = poisson.pmf(6, mu=calcio.away_goals_fact, loc=0)

home_win_results = [
                    (1, 0), (2, 0), (2, 1), (3, 0), (3, 1), (3, 2), 
                    (4, 0), (4, 1), (4, 2), (4, 3), 
                    (5, 0), (5, 1), (5, 2), (5, 3), (5, 4),
                    (6, 0), (6, 1), (6, 2), (6, 3), (6, 4), (6, 5)
                    ]
draw_results = [
                    (0, 0), (1, 1), (2, 2), (3, 3), (4, 4), (5, 5), (6, 6)
                    ]
away_win_results = [
                    (0, 1), (0, 2), (1, 2), (0, 3), (1, 3), (2, 3), 
                    (0, 4), (1, 4), (2, 4), (3, 4), 
                    (0, 5), (1, 5), (2, 5), (3, 5), (4, 5),
                    (0, 6), (1, 6), (2, 6), (3, 6), (4, 6), (5, 6)
                    ]

calcio['f_home_win'] = 0
for res in home_win_results:
    calcio['f_home_win'] = calcio['f_home_win'] + calcio['f_home_' + str(res[0])]*calcio['f_away_' + str(res[1])]

calcio['f_draw'] = 0
for res in draw_results:
    calcio['f_draw'] = calcio['f_draw'] + calcio['f_home_' + str(res[0])]*calcio['f_away_' + str(res[1])]

calcio['f_away_win'] = 0
for res in away_win_results:
    calcio['f_away_win'] = calcio['f_away_win'] + calcio['f_home_' + str(res[0])]*calcio['f_away_' + str(res[1])]

calcio['f_prob_sum'] = calcio['f_home_win'] + calcio['f_draw'] + calcio['f_away_win']
calcio['fRP1'] = calcio.f_home_win / calcio.f_prob_sum
calcio['fRPD'] = calcio.f_draw / calcio.f_prob_sum
calcio['fRP2'] = calcio.f_away_win / calcio.f_prob_sum

calcio['f_prob_sum'] = calcio['f_home_win'] + calcio['f_draw'] + calcio['f_away_win']
calcio['fRP1'] = calcio.f_home_win / calcio.f_prob_sum
calcio['fRPD'] = calcio.f_draw / calcio.f_prob_sum
calcio['fRP2'] = calcio.f_away_win / calcio.f_prob_sum

calcio['coeff_fact'] = calcio.R1-calcio.fRP1
calcio['coeff_fact_sign'] = 1
calcio.loc[calcio.coeff_fact <= 0, 'coeff_fact_sign'] = -1

In [ ]:
calcio['coeff_fact_sign'].sum()

In [ ]:
calcio.head()

<img src="https://s4.gifyu.com/images/animation_200_knjec20j.gif" width="200" height="200">

### **Готовим датафрейм для бэктестинга**
Делаем подобие таймфрейма, чтобы можно было бэктестить на готовом фреймворке для трейдинга.
- прибавляем коэффициент - 1 к предыдущему значению бара, если хозяева выиграли, вычитаем 1, если проиграли
- потом из предикшена делаем сигнал для бэктестинга.

In [ ]:
from sklearn.decomposition import PCA
import numpy as np
from datetime import date, timedelta

In [ ]:
calcio_backtesting = calcio[['year', 'week', 'team1_win', 'K1']].copy()
calcio_backtesting.year = '20' + calcio_backtesting.year.str.split('/').str[0]
#calcio_backtesting['predicted'] = list(calcio_predicted.Label)
calcio_backtesting = calcio_backtesting.append(pd.Series(
                                                        list(calcio_backtesting.iloc[-1]), 
                                                        index=calcio_backtesting.columns,
                                                        name=calcio_backtesting.index[-1] +1
                                                        ))
date_list = []
yr = 0
off = 0
for cnt, row in enumerate(zip(calcio_backtesting.year, calcio_backtesting.week)):
    if int(row[0]) > yr:
        yr = int(row[0])
        off = 0
    
    date_list.append(date(int(row[0]), 1, 1) + timedelta(days=1) * off) 
    off +=1

calcio_backtesting['Date'] = date_list
calcio_backtesting['Date'] = pd.to_datetime(calcio_backtesting['Date'])
calcio_backtesting.K1 = calcio_backtesting.K1 - 1
calcio_backtesting['Open'] = calcio_backtesting.team1_win * calcio_backtesting.K1
calcio_backtesting.Open = calcio_backtesting.Open.apply(lambda x: -1 if x < 0 else x).cumsum().shift(1).fillna(0) +100
calcio_backtesting['High'] = calcio_backtesting['Low'] = calcio_backtesting['Close'] = calcio_backtesting.Open

In [ ]:
calcio_backtesting.head()

In [ ]:
from backtesting import Backtest, Strategy
from bokeh.io import output_notebook

In [ ]:
def Kf(array):
    return pd.Series(array)

In [ ]:
class optimStrategy(Strategy):
#Стратегия только на покупку
    def init(self):        
        # Precompute value
        self.vStrat = self.I(Kf, self.data.predicted)       
        
    def next(self):
        if not self.position:        
            if self.vStrat == 1:
                self.buy(size = 1) # покупаем на условный 1$, здесь можной подумать над стратегией размера ставок
        else:                      # размер можно увеличивать в зависимости от уверенности модели
            if self.position.is_long:
                if self.vStrat == 0: # в отличие от трейдинга, сигнал на выход должен быть обратным сигналом входа
                    self.position.close()

<img src="https://s4.gifyu.com/images/animation_200_knjemytt.gif" width="200" height="200">

# **Готовим модель**
1. Берем результыты 5 последних матчей и отдельно 3 последних (домашних, выездных)
2. для каждой команды берём 2 первых компоненты PCA
3. добавляем верятность букмекера
4. на выход подаём переоценена ли вароятность букмекера или нет

In [ ]:
home_reduced = PCA(n_components = 2).fit_transform(
calcio[[
        'team1_back_1g_goals', 'team1_back_2g_goals', 'team1_back_3g_goals', 'team1_back_4g_goals', 'team1_back_5g_goals',
        'team1_back_1g_goals_home', 'team1_back_2g_goals_home', 'team1_back_3g_goals_home', 
        'team1_back_1g_concs', 'team1_back_2g_concs', 'team1_back_3g_concs', 'team1_back_4g_concs', 'team1_back_5g_concs',
        'team1_back_1g_concs_home', 'team1_back_2g_concs_home', 'team1_back_3g_concs_home'
        ]]
)

away_reduced = PCA(n_components = 2).fit_transform(
calcio[[
        'team2_back_1g_goals', 'team2_back_2g_goals', 'team2_back_3g_goals', 'team2_back_4g_goals', 'team2_back_5g_goals',
        'team2_back_1g_goals_away', 'team2_back_2g_goals_away', 'team2_back_3g_goals_away', 
        'team2_back_1g_concs', 'team2_back_2g_concs', 'team2_back_3g_concs', 'team2_back_4g_concs', 'team2_back_5g_concs',
        'team2_back_1g_concs_away', 'team2_back_2g_concs_away', 'team2_back_3g_concs_away'
        ]]
)

In [ ]:
calcio_pca = pd.DataFrame(np.concatenate((home_reduced, away_reduced), axis = 1), 
             columns=[
                 'home_pca1', 'home_pca2', 
                 'away_pca1', 'away_pca2',
                      ])

In [ ]:
calcio_pca['R1'] = calcio.R1.values
calcio_pca['coeff_fact_sign'] = pd.get_dummies(calcio.coeff_fact_sign).iloc[:,0].reset_index(drop = True)

<img src="https://s4.gifyu.com/images/animation_200_knjetzgi.gif" width="200" height="200">

# **Кастомная метрика**

In [ ]:
def single_instance_metric(row):
    if row['y_true'] == 1 and row['y_pred'] == 1: # False Positive
        return 1
    elif row['y_true'] == 0 and row['y_pred'] == 1: # False Negative
        return -1
    else: # Correct Predictions
        return 0
        
def calcio_score(y_true, y_pred):
    df = pd.DataFrame({'y_true':y_true, 'y_pred':y_pred})
    df['metric'] = df.apply(single_instance_metric, axis=1)
    return np.sum(df['metric'].values)

In [ ]:
from imblearn.over_sampling import *
adasyn1 = ADASYN(sampling_strategy='minority')

In [ ]:
adasyn1

In [ ]:
from pycaret.classification import *

In [ ]:
exp_clf102 = setup(
                  data = calcio_pca, 
                  target = 'coeff_fact_sign', session_id=123,
                  #combine_rare_levels = False, rare_level_threshold = 0.05,
                  remove_multicollinearity = False, multicollinearity_threshold = 0.95, 
                  #ignore_features = [],
                  train_size = 0.91,
                  data_split_shuffle = False,
                  normalize = False, 
                  transformation = True,
                  pca = False,
                  #pca_components = 2,
                  #fix_imbalance=True,
                  #fix_imbalance_method= adasyn1,
                  log_experiment = False, 
                  experiment_name = 'calcio1',
                  silent = True 
                   )

In [ ]:
add_metric(
    id='calcio_score',
    name='Calcio Score',
    score_func=calcio_score,
    target='pred',
    greater_is_better=True)

In [ ]:
get_metrics()

In [ ]:
best = compare_models(
                      fold=10,
                      sort='Prec.'
                      )

<img src="https://s4.gifyu.com/images/animation_200_knjf8nl6.gif" width="200" height="200">

# **Варим модели**

# ada

In [ ]:
ada = create_model('ada', fold =10)

In [ ]:
tuned_ada = tune_model(
                      ada, optimize = 'Calcio Score', 
                      fold = 10, n_iter = 25, 
                      search_library = 'optuna', 
                      search_algorithm = 'random'
                     )

In [ ]:
plot_model(tuned_lda, plot = 'confusion_matrix')

In [ ]:
ensembled_ada = ensemble_model(tuned_ada, optimize = 'Calcio Score', fold = 10, n_estimators = 25, method='Boosting')

In [ ]:
plot_model(ensembled_ada, plot = 'confusion_matrix')

### бэктестим ada

In [ ]:
predictions = predict_model(ensembled_ada, data=calcio_pca)
prediction_list = list(predictions.Label)
prediction_list.append(prediction_list[-1])
calcio_backtesting['predicted'] = prediction_list
optbt = Backtest(
                 calcio_backtesting[['Date', 'Open', 'High', 'Low', 'Close', 'predicted']].set_index('Date')[2235:], #(тест/трейн)
                 optimStrategy, 
                 cash=200, 
                 commission=.0, 
                 exclusive_orders = True)
stats = optbt.run()
print(stats)

### рисуем результаты бэктеста

In [ ]:
output_notebook()
optbt.plot(plot_width=1200, plot_volume = False, plot_drawdown = True)

# RF

In [ ]:
rf = create_model('rf', fold = 10)

In [ ]:
tuned_rf = tune_model(rf, optimize = 'Prec.', 
                           fold = 10, n_iter = 25, search_library = 'optuna', 
                           search_algorithm = 'tpe')

In [ ]:
plot_model(tuned_rf, plot = 'confusion_matrix')

In [ ]:
ensembled_rf = ensemble_model(tuned_rf, optimize = 'Calcio Score', fold = 10, n_estimators = 25, method='Boosting')

### бэктестим RF

In [ ]:
predictions = predict_model(tuned_rf, data=calcio_pca)
prediction_list = list(predictions.Label)
prediction_list.append(prediction_list[-1])
calcio_backtesting['predicted'] = prediction_list
optbt = Backtest(
                 calcio_backtesting[['Date', 'Open', 'High', 'Low', 'Close', 'predicted']].set_index('Date')[2235:],
                 optimStrategy, 
                 cash=200, 
                 commission=.0, 
                 exclusive_orders = True)
stats = optbt.run()
print(stats)

### рисуем результаты бэктеста

In [ ]:
output_notebook()
optbt.plot(plot_width=1200, plot_volume = False, plot_drawdown = True)

# GBC

In [ ]:
gbc = create_model('gbc', fold = 10)

In [ ]:
tuned_gbc = tune_model(gbc, optimize = 'Prec.', 
                           fold = 10, n_iter = 25, search_library = 'optuna', 
                           search_algorithm = 'tpe')

In [ ]:
plot_model(tuned_gbc, plot = 'confusion_matrix')

In [ ]:
ensembled_gbc = ensemble_model(tuned_gbc, optimize = 'Calcio Score', fold = 10, n_estimators = 25, method='Bagging')

In [ ]:
plot_model(ensembled_gbc, plot = 'confusion_matrix')

### бэктестим GBC

In [ ]:
predictions = predict_model(ensembled_gbc, data=calcio_pca)
prediction_list = list(predictions.Label)
prediction_list.append(prediction_list[-1])
calcio_backtesting['predicted'] = prediction_list
optbt = Backtest(
                 calcio_backtesting[['Date', 'Open', 'High', 'Low', 'Close', 'predicted']].set_index('Date')[2235:],
                 optimStrategy, 
                 cash=200, 
                 commission=.0, 
                 exclusive_orders = True)
stats = optbt.run()
print(stats)

### рисуем результаты бэктеста

In [ ]:
output_notebook()
optbt.plot(plot_width=1200, plot_volume = False, plot_drawdown = True)

In [ ]:
# blend individual models
blender = blend_models(estimator_list = [
                                         tuned_ada,
                                         tuned_gbc,
                                         tuned_rf
                                         ], 
                       optimize = 'Calcio Score', fold = 10
                       )

In [ ]:
plot_model(blender, plot = 'confusion_matrix')

### бэктестим блендер

In [ ]:
predictions = predict_model(blender, data=calcio_pca)
prediction_list = list(predictions.Label)
prediction_list.append(prediction_list[-1])
calcio_backtesting['predicted'] = prediction_list
optbt = Backtest(
                 calcio_backtesting[['Date', 'Open', 'High', 'Low', 'Close', 'predicted']].set_index('Date')[2235:],
                 optimStrategy, 
                 cash=200, 
                 commission=.0, 
                 exclusive_orders = True)
stats = optbt.run()
print(stats)

In [ ]:
output_notebook()
optbt.plot(plot_width=1200, plot_volume = False, plot_drawdown = True)

In [ ]:
stacker = stack_models(estimator_list = [
                                         tuned_ada,
                                         tuned_gbc,
                                         tuned_rf
                                         ], 
                       optimize = 'F1', fold =10,
                       meta_model = ada
                       )

### бэктестим стэкинг

In [ ]:
predictions = predict_model(blender, data=calcio_pca)
prediction_list = list(predictions.Label)
prediction_list.append(prediction_list[-1])
calcio_backtesting['predicted'] = prediction_list
optbt = Backtest(
                 calcio_backtesting[['Date', 'Open', 'High', 'Low', 'Close', 'predicted']].set_index('Date')[2235:],
                 optimStrategy, 
                 cash=200, 
                 commission=.0, 
                 exclusive_orders = True)
stats = optbt.run()
print(stats)

In [ ]:
output_notebook()
optbt.plot(plot_width=1200, plot_volume = False, plot_drawdown = True)

In [ ]:
print(stacker)

<img src="https://s4.gifyu.com/images/animation_200_knjgf0c3.gif" width="200" height="200">

# Что бы еще пригодилось:
1. Менеджмент (стратегия) ставок. Многое можно выжать регулирую размер ставки
2. Можно попробовать нейроночки и засунуть бэктестинг в callback